# SYCL devices

In [66]:
%%writefile compute.cpp
#include <sycl/sycl.hpp>
#include <vector>

int main(int argc, char **argv) {
    const int N = 2;
    
    auto myQueue = sycl::queue{sycl::gpu_selector_v};
    std::cout << "Running on "
              << myQueue.get_device().get_info<sycl::info::device::name>()
              << "\n";

    int *dataA = sycl::malloc_shared<int>(N, myQueue);
    int *dataB = sycl::malloc_shared<int>(N, myQueue);
    int *dataC = sycl::malloc_shared<int>(N, myQueue);

    // STEP 3 : Asign values to vectors
    for (int i = 0; i < N; i++) {
        dataA[i] = 1;
        dataB[i] = 2;
        dataC[i] = 0;
    }
    
    // STEP 4 : Proceed with the calculation
    myQueue.parallel_for(sycl::range<1>(N), [=](sycl::id<1> i) {
      dataC[i] = dataA[i] + dataB[i];
    }).wait();

    // Print out the result of vector add
    int indices[]{0, 1, 2, (static_cast<int>(N) - 1)};
    constexpr size_t indices_size = sizeof(indices) / sizeof(int);
    for (int i = 0; i < indices_size; i++) {
        int j = indices[i];
        if (i == indices_size - 1) std::cout << "...\n";
            std::cout << "[" << j << "]: " << dataA[j] << " + " << dataB[j] << " = "
                      << dataC[j] << "\n";
        }

    sycl::free(dataA, myQueue);
    sycl::free(dataB, myQueue);
    sycl::free(dataC, myQueue);

    return 0;
}


Overwriting compute.cpp


In [67]:
%%writefile ./run-sycl.sh

#!/bin/bash -x
source /opt/intel/oneapi/setvars.sh > /dev/null 2>&1
icpx -fsycl compute.cpp
if [ $? -eq 0 ]; then ./a.out; fi

Overwriting ./run-sycl.sh


In [68]:
!chmod u+x ./run-sycl.sh &&./run-sycl.sh

Running on Intel(R) Graphics [0x46a6]
[0]: 1 + 2 = 3
[1]: 1 + 2 = 3
[2]: 0 + 0 = 0
...
[1]: 1 + 2 = 3
